In [1]:
!pip install openai google-genai

  Attempting uninstall: cachetools
    Found existing installation: cachetools 6.2.1
    Uninstalling cachetools-6.2.1:
      Successfully uninstalled cachetools-6.2.1


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
"""
multi_agent_gym_projects.py

Updated: Replaced the simulated LLM with a clear example showing how to call OpenAI's API (instructions only — code will not run here unless you provide an API key and install the OpenAI Python package).

Also: extended the WorkoutAgent to produce detailed exercises with sets/reps/load suggestions based on goal.

How to enable OpenAI in this file:
  1. pip install openai google-genai
  2. export OPENAI_API_KEY="sk-..." (or set as env var on Windows)
  3. Optionally change MODEL_NAME below to your preferred model (e.g., "gpt-4o", "gpt-4o-mini", or Chat Completions model).

Note: This file contains a safe example of integrating with OpenAI. It's written so you can run it locally or in Kaggle after you set the env var.
"""

'\nmulti_agent_gym_projects.py\n\nUpdated: Replaced the simulated LLM with a clear example showing how to call OpenAI\'s API (instructions only — code will not run here unless you provide an API key and install the OpenAI Python package).\n\nAlso: extended the WorkoutAgent to produce detailed exercises with sets/reps/load suggestions based on goal.\n\nHow to enable OpenAI in this file:\n  1. pip install openai google-genai\n  2. export OPENAI_API_KEY="sk-..." (or set as env var on Windows)\n  3. Optionally change MODEL_NAME below to your preferred model (e.g., "gpt-4o", "gpt-4o-mini", or Chat Completions model).\n\nNote: This file contains a safe example of integrating with OpenAI. It\'s written so you can run it locally or in Kaggle after you set the env var.\n'

In [4]:
from __future__ import annotations
import os
import random
import threading
from typing import List, Dict, Any, Optional, Tuple
import time
import logging
import uuid
# Optional OpenAI integration. This demonstrates how you'd replace the simulated LLM.
# The code will not run here unless you install openai and provide an API key.
# Install: pip install openai
# Environment variable: OPENAI_API_KEY
import openai
import google.generativeai as genai
from google import genai
from kaggle_secrets import UserSecretsClient
import json
import re
from typing import Tuple

In [5]:
user_secrets = UserSecretsClient()
GEMINI_API_KEY = user_secrets.get_secret("GEMINI_API_KEY")
MODEL_NAME = "gemini-2.5-flash"

In [6]:
# CONFIG
# MODEL_NAME = "gpt-4o-mini"  # Change to the model you want to use
# OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

In [7]:
# LLM wrapper: will call OpenAI if package & key are present, otherwise returns a deterministic fallback.
def call_openai_chat(prompt: str, model: str = MODEL_NAME, temperature: float = 0.6, max_tokens: int = 400) -> str:
    """
    Calls the OpenAI ChatCompletions endpoint (example). This function is provided as an implementation guide.

    Important: This demonstrates the request body shape for openai.ChatCompletion.create. Depending on the OpenAI SDK
    version you use, function names and parameters might differ slightly. Consult the official OpenAI Python docs for
    your installed SDK version.

    Replace usage with whichever OpenAI client or method you prefer.
    """
    if openai is None or not OPENAI_API_KEY:
        # Fallback deterministic reply when OpenAI isn't available
        return (
            "[LLM not configured] Provide an OPENAI_API_KEY and install the 'openai' package to enable real LLM calls.\n"
            + "Prompt received:\n" + prompt[:500]
        )

    # configure client
    openai.api_key = OPENAI_API_KEY

    # Example using ChatCompletion (classic SDK). Your environment or SDK version may use a different call.
    try:
        response = openai.ChatCompletion.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            temperature=temperature,
            max_tokens=max_tokens,
        )
        # Extract assistant reply
        if isinstance(response, dict):
            # typical structure
            return response["choices"][0]["message"]["content"].strip()
        else:
            # Some SDKs return objects with attributes
            return response.choices[0].message.content.strip()
    except Exception as e:
        return f"[OpenAI call failed] {e}"

In [8]:
def call_gemini(prompt: str, model: str = "gemini-2.5-flash", temperature: float = 0.6, max_tokens: int = 400) -> str:
    """
    Call Google Gemini via the Google GenAI SDK if available and configured.
    Falls back to raising RuntimeError if not configured (caller should catch and fallback).
    """
    if genai is None:
        raise RuntimeError("google-genai SDK not installed (pip install google-genai)")

    # Configure client: prefer explicit API key if provided, otherwise rely on ADC
    try:
        # Some examples use genai.configure(...) and others use client objects.
        # Attempt common patterns used in Google docs.
        if GEMINI_API_KEY:
            # older/newer SDKs vary; configure global API key if available
            try:
                genai.configure(api_key=GEMINI_API_KEY)
            except AttributeError:
                # some SDK variants expect client instantiation
                client = genai.Client(api_key=GEMINI_API_KEY)
            resp = client.models.generate_content(model=model, contents=prompt)
            # print("Gemini Call", resp)
            print("Gemini Call")
            return getattr(resp, "text", str(resp))
        # If configure not used, try client interface
        try:
            client = genai.Client()  # will use ADC if no API key
            resp = client.models.generate_content(model=model, contents=prompt)
            # resp.text is the usual attribute in examples
            return getattr(resp, "text", str(resp))
        except Exception:
            # fallback to older method using direct model object
            model_obj = genai.GenerativeModel(model)
            response = model_obj.generate_content(prompt)
            return getattr(response, "text", str(response))
    except Exception as e:
        # bubble up so caller can fall back to timed_llm_call/OpenAI fallback
        raise RuntimeError(f"Gemini call failed: {e}")

In [9]:
logging.basicConfig(level=logging.INFO, format="%(asctime)s %(levelname)s %(message)s")
logger = logging.getLogger("multi_agent_gym")

METRICS = {
    "llm_calls": 0,
    "errors": 0,
    "total_llm_time_s": 0.0,
    "sequential_flows": 0,
    "parallel_flows": 0,
    "loop_iterations": 0,
}


def print_metrics():
    logger.info("METRICS: %s", METRICS)

In [10]:
def timed_llm_call(prompt: str, model: str = MODEL_NAME, temperature: float = 0.6, max_tokens: int = 400) -> str:
    start = time.time()
    METRICS["llm_calls"] += 1
    call_id = uuid.uuid4().hex[:8]
    logger.info("LLM call %s started (len(prompt)=%d)", call_id, len(prompt))
    try:
        out = call_gemini(prompt, model=model, temperature=temperature, max_tokens=max_tokens)
        # print("Gemini Call", out)
        print("Gemini Call")
        return out
    except Exception:
        METRICS["errors"] += 1
        logger.exception("LLM call %s error", call_id)
        raise
    finally:
        elapsed = time.time() - start
        METRICS["total_llm_time_s"] += elapsed
        logger.info("LLM call %s finished in %.3fs", call_id, elapsed)

In [11]:
# ---------------------- Nutrition utilities ----------------------
def estimate_calories_for_meal(meal: Dict[str, Any]) -> float:
    """Basic calorie estimate by summing item estimates. In production, hook a nutrition DB.
    meal: {"items": [{"name": str, "qty": float, "unit": str, "kcal_per_unit": float}, ...]}
    """
    total = 0.0
    for it in meal.get("items", []):
        kcal = float(it.get("kcal_per_unit", 0.0)) * float(it.get("qty", 1.0))
        total += kcal
    return round(total, 2)


def sum_micronutrients(meals: List[Dict[str, Any]]) -> Dict[str, float]:
    totals: Dict[str, float] = {}
    for meal in meals:
        for micro, val in meal.get("micros", {}).items():
            totals[micro] = totals.get(micro, 0.0) + float(val)
    return {k: round(v, 2) for k, v in totals.items()}

In [12]:
# ---------------------- Sample utilities & main demo ----------------------
def sample_meals() -> List[Dict[str, Any]]:
    return [
        {
            "name": "Breakfast",
            "items": [{"name": "oats", "qty": 1, "unit": "cup", "kcal_per_unit": 300}, {"name": "banana", "qty": 1, "unit": "piece", "kcal_per_unit": 100}],
            "micros": {"protein_g": 12, "vitamin_c_mg": 10},
        },
        {
            "name": "Lunch",
            "items": [{"name": "chicken_breast", "qty": 200, "unit": "g", "kcal_per_unit": 1.65}],
            "micros": {"protein_g": 45, "vitamin_c_mg": 5},
        },
    ]


def incremental_meal_generator(iteration: int) -> List[Dict[str, Any]]:
    # Example generator: small variations each iteration
    base = sample_meals()
    # add a snack that grows each iteration
    base.append({
        "name": f"Snack_{iteration}",
        "items": [{"name": "almonds", "qty": 30 + iteration * 10, "unit": "g", "kcal_per_unit": 6}],
        "micros": {"protein_g": 6 + iteration * 1},
    })
    return base

In [13]:
def compact_context(messages: List[str], summarizer=timed_llm_call, max_len_chars: int = 2000) -> str:
    joined = "\n".join(messages)
    if len(joined) <= max_len_chars:
        return joined

    prompt = (
        "You are a context compaction assistant. Summarize the following "
        "conversation/messages into a concise 3-bullet summary:\n\n"
        + joined[:10000]
    )
    try:
        summary = summarizer(prompt, temperature=0.2, max_tokens=300)
        compacted = "[COMPACT_SUMMARY]\n" + summary.strip()
        logger.info("Context compacted from %d chars to %d chars", len(joined), len(compacted))
        return compacted
    except Exception:
        logger.exception("Context compaction failed; returning truncated original")
        return joined[-max_len_chars:]

In [14]:
# ---------------------- Agents ----------------------
class BaseAgent:
    def run(self, *args, **kwargs):
        raise NotImplementedError

In [15]:
class FoodAgent(BaseAgent):
    def analyze_meals(self, meals: List[Dict[str, Any]]) -> Dict[str, Any]:
        total_calories = sum(estimate_calories_for_meal(m) for m in meals)
        micros = sum_micronutrients(meals)

        messages = [
            "Nutrition assistant context:",
            f"Meals data: {meals}",
            f"Micronutrients summary: {micros}",
            f"Total calories (est): {total_calories}",
            "Task: Provide a short natural-language summary and 3 practical tips."
        ]

        compact_prompt = compact_context(messages)
        llm_reply = timed_llm_call(compact_prompt)
        return {
            "total_calories": total_calories,
            "micronutrients": micros,
            "llm_summary": llm_reply,
        }

    def run(self, meals: List[Dict[str, Any]]):
        return self.analyze_meals(meals)

In [16]:
# Extended WorkoutAgent: returns exercises with reps/sets/load suggestions.
class WorkoutAgent(BaseAgent):
    # A small library of exercises mapped to muscle groups and equipment
    EXERCISE_DB = {
        "push": [
            {"name": "Barbell Bench Press", "equipment": "barbell", "primary": "chest"},
            {"name": "Dumbbell Shoulder Press", "equipment": "dumbbells", "primary": "shoulders"},
            {"name": "Push-up", "equipment": "bodyweight", "primary": "chest"},
        ],
        "pull": [
            {"name": "Barbell Row", "equipment": "barbell", "primary": "back"},
            {"name": "Pull-up", "equipment": "bodyweight", "primary": "back"},
            {"name": "Dumbbell Curl", "equipment": "dumbbells", "primary": "biceps"},
        ],
        "legs": [
            {"name": "Back Squat", "equipment": "barbell", "primary": "quads"},
            {"name": "Romanian Deadlift", "equipment": "barbell", "primary": "hamstrings"},
            {"name": "Lunges", "equipment": "bodyweight", "primary": "quads"},
        ],
        "core": [
            {"name": "Plank", "equipment": "bodyweight", "primary": "core"},
            {"name": "Hanging Leg Raise", "equipment": "bodyweight", "primary": "core"},
        ],
    }

    # Simple heuristics for reps/sets/load by training goal
    GOAL_PARAMS = {
        "strength": {"sets": [3, 5], "reps": [3, 6], "intensity_pct": [80, 90]},  # % of 1RM
        "hypertrophy": {"sets": [3, 4], "reps": [6, 12], "intensity_pct": [65, 80]},
        "endurance": {"sets": [2, 3], "reps": [12, 20], "intensity_pct": [40, 65]},
        "general": {"sets": [3, 4], "reps": [8, 12], "intensity_pct": [60, 75]},
    }

    def suggest_load(self, user_profile: Dict[str, Any], intensity_pct: float, exercise_name: str) -> Optional[float]:
        """
        Suggest a simple absolute load (kg) for barbell/dumbbell lifts using an optional 1RM estimate in user_profile.
        If no 1RM is provided, return None to indicate 'RPE-style' guidance.
        """
        one_rm_map = user_profile.get("1rm", {})  # e.g., {"Barbell Bench Press": 100}
        if exercise_name in one_rm_map:
            one_rm = float(one_rm_map[exercise_name])
            return round(one_rm * (intensity_pct / 100.0), 1)
        return None

    def pick_exercises_for_day(self, focus: str, num_exercises: int = 4) -> List[Dict[str, Any]]:
        pool = []
        # Build pool from focus and some auxiliary groups
        if focus == "full_body":
            pool = self.EXERCISE_DB["push"] + self.EXERCISE_DB["pull"] + self.EXERCISE_DB["legs"] + self.EXERCISE_DB["core"]
        else:
            pool = self.EXERCISE_DB.get(focus, []) + self.EXERCISE_DB.get("core", [])

        chosen = random.sample(pool, min(len(pool), num_exercises))
        return chosen

    def create_exercise_detail(self, exercise: Dict[str, Any], goal: str, user_profile: Dict[str, Any]) -> Dict[str, Any]:
        params = self.GOAL_PARAMS.get(goal, self.GOAL_PARAMS["general"])
        sets = random.randint(params["sets"][0], params["sets"][1])
        reps = random.randint(params["reps"][0], params["reps"][1])
        intensity_pct = random.uniform(params["intensity_pct"][0], params["intensity_pct"][1])
        load = self.suggest_load(user_profile, intensity_pct, exercise["name"]) if exercise["equipment"] in ("barbell", "dumbbells") else None

        guidance = []
        if load:
            guidance.append(f"Target load ~ {load} kg ({int(round(intensity_pct))}% 1RM)")
        else:
            guidance.append(f"Target intensity: ~{int(round(intensity_pct))}% effort; choose a weight that allows you to hit {reps} reps with good form.")

        # Simple progression suggestion
        guidance.append("Progression: add 2.5-5% load when you hit the top of the rep range for 2 consecutive sessions.")

        return {
            "name": exercise["name"],
            "primary": exercise.get("primary"),
            "equipment": exercise.get("equipment"),
            "sets": sets,
            "reps": reps,
            "recommended_load_kg": load,
            "notes": guidance,
        }

    def create_5_day_plan(self, goal: str = "general", user_profile: Optional[Dict[str, Any]] = None) -> Dict[str, Any]:
        if user_profile is None:
            user_profile = {}

        # Simple 5-day split depending on goal
        if goal == "strength":
            split = ["push", "pull", "legs", "push", "pull"]
        elif goal == "hypertrophy":
            split = ["push", "pull", "legs", "full_body", "legs"]
        else:  # general or endurance
            split = ["push", "pull", "legs", "core", "full_body"]

        week_plan = {}
        for i, day_focus in enumerate(split, start=1):
            exercises = self.pick_exercises_for_day(day_focus, num_exercises=4)
            detailed = [self.create_exercise_detail(ex, goal, user_profile) for ex in exercises]
            week_plan[f"Day {i} - {day_focus}"] = detailed

        # Optionally ask LLM for a high-level program summary
        messages = [
            f"User goal: {goal}",
            f"User profile: {user_profile}",
            "Program plan (detailed):",
            str(week_plan),
            "Task: Create a concise 3-bullet summary for this 5-day program."
        ]
        compact_prompt = compact_context(messages)
        llm_summary = None
        # print(genai)
        try:
            if genai is not None and (GEMINI_API_KEY or os.getenv("GOOGLE_APPLICATION_CREDENTIALS")):
                # prime metrics/logging if you want (we reuse existing logger/METRICS)
                logger.info("Attempting Gemini call for WorkoutAgent summary")
                llm_summary = call_gemini(compact_prompt, model="gemini-2.5-flash", temperature=0.2, max_tokens=300)
                logger.info("Gemini summary received (len=%d)", len(llm_summary or ""))
        except Exception as e:
            logger.warning("Gemini call failed or not configured: %s", e)
        
        # If Gemini not configured or call failed, use timed_llm_call() (OpenAI or fallback)
        if not llm_summary:
            logger.info("Falling back to timed_llm_call for WorkoutAgent summary")
            llm_summary = timed_llm_call(compact_prompt)
        
        return {"plan": week_plan, "summary": llm_summary}

    def run(self, goal: str = "general", user_profile: Optional[Dict[str, Any]] = None):
        return self.create_5_day_plan(goal, user_profile)


In [17]:
### ReminderAgent - simple agent for reminders (drink water, stretch, etc.)
class ReminderAgent(BaseAgent):
    def remind_water(self, user_profile: Dict[str, Any]) -> Dict[str, Any]:
        # Very simple decision: frequency based on goal or basic rule
        goal = user_profile.get("goal", "general")
        if goal == "endurance":
            freq = "every 30-45 minutes"
        elif goal == "strength":
            freq = "every 45-60 minutes"
        else:
            freq = "every 60-90 minutes"
        msg = f"Hydration reminder: Aim to drink water {freq} while training and throughout the day."
        return {"reminder": msg, "recommended_frequency": freq}

    def remind_stretch(self) -> Dict[str, Any]:
        msg = "Take a 3–5 minute mobility/stretch break between heavy sets or every hour during the day."
        return {"reminder": msg}

In [18]:
class EvaluationAgent(BaseAgent):
    def evaluate_calorie_plan(self, calories: float, target_range: Tuple[float, float]) -> Dict[str, Any]:
        low, high = target_range
        score = 100 - max(0, abs((calories - (low + high) / 2))) / ((high - low) / 2 + 1) * 50
        verdict = "within target" if low <= calories <= high else "outside target"
        return {"score": round(score, 1), "verdict": verdict}

    def evaluate_workout(self, plan: Dict[str, Any]) -> Dict[str, Any]:
        # Basic heuristics: count exercise variety and average sets/reps
        all_ex = []
        for day, exs in plan.items():
            all_ex.extend(exs)
        variety = len({e["name"] for e in all_ex})
        avg_sets = sum(e["sets"] for e in all_ex) / max(1, len(all_ex))
        return {"variety": variety, "avg_sets": round(avg_sets, 1)}

    def run(self, items: Dict[str, Any]):
        # Example combined evaluation
        cal_eval = self.evaluate_calorie_plan(items.get("calories", 0), items.get("target_range", (1800, 2400)))
        workout_eval = self.evaluate_workout(items.get("workout_plan", {}))
        return {"cal_eval": cal_eval, "workout_eval": workout_eval}

In [19]:
# ---------------------- Orchestrator ----------------------
class Orchestrator:
    def __init__(self):
        self.food_agent = FoodAgent()
        self.workout_agent = WorkoutAgent()
        self.eval_agent = EvaluationAgent()
        self.reminder_agent = ReminderAgent()  # ADDED

    def sequential_flow(self, meals: List[Dict[str, Any]], user_profile: Dict[str, Any]):
        """
        Sequential flow: Nutrition -> Workout Plan -> Evaluation
        (keeps the original pipeline behavior)
        """
        METRICS["sequential_flows"] += 1
        food_res = self.food_agent.run(meals)
        workout_res = self.workout_agent.run(user_profile.get("goal", "general"), user_profile)
        evaluation = self.eval_agent.run({
            "calories": food_res["total_calories"],
            "workout_plan": workout_res["plan"],
            "target_range": user_profile.get("calorie_target", (1800, 2400))
        })
        return {"food": food_res, "workout": workout_res, "evaluation": evaluation}

    def parallel_flow(self, meals: List[Dict[str, Any]], user_profile: Dict[str, Any]):
        """
        Parallel flow example: Run nutrition analysis and hydration reminder concurrently,
        but give the workout plan afterwards. Demonstrates different capabilities run in parallel.
        """
        METRICS["parallel_flows"] += 1
        food_res = {}
        reminder_res = {}
        workout_res = {}

        def run_food():
            nonlocal food_res
            food_res = self.food_agent.run(meals)

        def run_reminder():
            nonlocal reminder_res
            reminder_res = self.reminder_agent.remind_water(user_profile)

        # run food analysis and reminder concurrently (two different agent types)
        t1 = threading.Thread(target=run_food)
        t2 = threading.Thread(target=run_reminder)
        t1.start(); t2.start(); t1.join(); t2.join()

        # after parallel tasks finish, produce a workout plan (separate agent)
        workout_res = self.workout_agent.run(user_profile.get("goal", "general"), user_profile)

        evaluation = self.eval_agent.run({
            "calories": food_res.get("total_calories", 0),
            "workout_plan": workout_res["plan"],
            "target_range": user_profile.get("calorie_target", (1800, 2400))
        })
        return {"food": food_res, "reminder": reminder_res, "workout": workout_res, "evaluation": evaluation}

    def loop_agent(self, meal_generator, user_profile: Dict[str, Any], iterations: int = 3):
        """
        Loop agent example that iteratively refines the user's plan.
        On each iteration it:
         - evaluates current plan,
         - optionally tweaks user_profile (simple heuristic),
         - and requests a new plan.
        """
        history = []
        for i in range(iterations):
            METRICS["loop_iterations"] += 1
            meals = meal_generator(i)
            res = self.sequential_flow(meals, user_profile)
            # Simple self-improvement heuristic:
            # if evaluation indicates calories are outside target, nudge the calorie target
            cal_eval = res["evaluation"]["cal_eval"]
            low, high = user_profile.get("calorie_target", (1800, 2400))
            if cal_eval["verdict"] == "outside target":
                # nudge midpoint towards observed calories
                observed = res["food"]["total_calories"]
                midpoint = (low + high) / 2
                shift = (observed - midpoint) * 0.3
                new_low, new_high = int(low + shift), int(high + shift)
                user_profile["calorie_target"] = (max(1200, new_low), max(new_low + 200, new_high))
                logger.info("Loop agent adjusted calorie_target to %s", user_profile["calorie_target"])
            history.append(res)
        return history

    def run_tools(self, tools: List[str], meals: List[Dict[str, Any]], user_profile: Dict[str, Any]):
        """
        Dispatcher: run a list of tool names and return aggregated results.
        Supported tool names: 'nutrition', 'workout', 'evaluate', 'reminder_water', 'reminder_stretch', 'sequential', 'parallel', 'loop'
        """
        results = {}
        for t in tools:
            if t == "nutrition":
                results["nutrition"] = self.food_agent.run(meals)
            elif t == "workout":
                results["workout"] = self.workout_agent.run(user_profile.get("goal", "general"), user_profile)
            elif t == "evaluate":
                # requires prior nutrition and workout results; attempt to compute if missing
                calib = results.get("nutrition", {}).get("total_calories", 0)
                plan = results.get("workout", {}).get("plan", {})
                results["evaluation"] = self.eval_agent.run({"calories": calib, "workout_plan": plan, "target_range": user_profile.get("calorie_target", (1800, 2400))})
            elif t == "reminder_water":
                results["reminder_water"] = self.reminder_agent.remind_water(user_profile)
            elif t == "reminder_stretch":
                results["reminder_stretch"] = self.reminder_agent.remind_stretch()
            elif t == "sequential":
                results["sequential"] = self.sequential_flow(meals, user_profile)
            elif t == "parallel":
                results["parallel"] = self.parallel_flow(meals, user_profile)
            elif t == "loop":
                results["loop"] = self.loop_agent(lambda i: meals, user_profile, iterations=2)
            else:
                results[t] = {"error": f"Unknown tool '{t}'"}
        return results


In [20]:
### decide_tools_from_prompt - asks the LLM which tools to run given a user prompt.
def decide_tools_from_prompt(prompt: str) -> List[str]:
    """
    Ask the LLM to map a user prompt to a small list of tool keys.
    Returns a list like ['nutrition', 'workout', 'reminder_water'].
    Uses timed_llm_call(); falls back to a keyword heuristic if LLM not configured.
    """
    # small instruction prompt enumerating available tools
    tools=["nutrition", "workout", "evaluate", "reminder_water", "reminder_stretch", "sequential", "parallel", "loop"]
    instruction = (
        "You are an assistant that maps a user request to a list of tool names. "
        f"Available tool names: {tools}\n\n"
        f"User request: {prompt}\n\n"
        "Return ONLY a comma-separated list of the tool names (no other text)."
    )
    try:
        resp = timed_llm_call(instruction, temperature=0.0, max_tokens=80)
        # split on commas and return allowed tools only
        candidates = [s.strip().lower() for s in resp.replace("\n", ",").split(",") if s.strip()]
        allowed = {"nutrition", "workout", "evaluate", "reminder_water", "reminder_stretch", "sequential", "parallel", "loop"}
        tools = [c for c in candidates if c in allowed]
        if tools:
            logger.info("LLM decided tools: %s", tools)
            return tools
    except Exception as e:
        logger.warning("Tool decision LLM failed: %s", e)

    # fallback keyword heuristic
    kw = prompt.lower()
    tools = []
    if any(x in kw for x in ["meal", "calorie", "nutrition", "eat"]):
        tools.append("nutrition")
    if any(x in kw for x in ["workout", "exercise", "plan", "training"]):
        tools.append("workout")
    if any(x in kw for x in ["evaluate", "score", "assess"]):
        tools.append("evaluate")
    if any(x in kw for x in ["water", "hydrate", "drink"]):
        tools.append("reminder_water")
    if any(x in kw for x in ["stretch", "mobility"]):
        tools.append("reminder_stretch")
    if "loop" in kw or "iterate" in kw or "improve" in kw:
        tools.append("loop")
    if not tools:
        # by default propose the sequential pipeline
        tools = ["sequential"]
    logger.info("Fallback heuristic decided tools: %s", tools)
    return tools

In [21]:
### LLM-guided natural-language -> user_profile extractor
def _extract_json_substring(text: str) -> str:
    """Return first {...} JSON-like substring if present, else full text."""
    m = re.search(r"\{.*\}", text, flags=re.DOTALL)
    return m.group(0) if m else text

def load_user_profile_from_prompt(nl_text: str, prefer_llm: bool = True, llm_timeout: int = 10) -> dict:
    """
    Convert a natural-language description of the user into a structured user_profile:
      {
        "goal": "hypertrophy" | "strength" | "endurance" | "general",
        "calorie_target": (low_float, high_float),
        "1rm": {"Exercise Name": float, ...}
      }

    Behavior:
      - If timed_llm_call() is available it will be used (preferred).
      - If LLM fails or is not available, a regex-based heuristic extractor runs as fallback.
      - The function is robust to the LLM returning plaintext or a JSON object — it attempts to parse JSON out of the reply.
    """

    # 1) Build an instruction prompt that asks for JSON only
    instruction = (
        "Extract a structured user profile from this user description.\n\n"
        "Output MUST be valid JSON only (no extra commentary). Schema:\n"
        '{\n'
        '  "goal": "<strength|hypertrophy|endurance|general>",\n'
        '  "calorie_target": [low_number, high_number],\n'
        '  "1rm": {"Exercise Name": weight_number, ...}\n'
        '}\n\n'
        "If a value is unknown, use null for numbers or an empty object for 1rm. "
        "If a value is unknown for calorie_target, assign a value according to user needs. "
        "Normalize the goal to one of: strength, hypertrophy, endurance, general.\n\n"
        f"User description:\n\"\"\"\n{nl_text}\n\"\"\"\n"
        "Return only the JSON object."
    )

    # 2) Try to call the LLM 
    llm_response = None
    try:
        print("Gemini Call from load_user_profile_from_prompt")
        llm_response = timed_llm_call(instruction, temperature=0.0, max_tokens=300)
    except Exception:
        llm_response = None

    # 3) If we got an LLM response, try to parse JSON substring
    if llm_response:
        try:
            candidate = _extract_json_substring(llm_response)
            parsed = json.loads(candidate)
            # Basic normalization / validation:
            # normalize goal
            if "goal" in parsed:
                g = parsed["goal"].strip().lower()
                if g.startswith("str"):
                    parsed["goal"] = "strength"
                elif "hyper" in g:
                    parsed["goal"] = "hypertrophy"
                elif "endur" in g:
                    parsed["goal"] = "endurance"
                else:
                    parsed["goal"] = "general"
            else:
                parsed.setdefault("goal", "general")

            # calorie_target -> tuple
            if "calorie_target" in parsed:
                low = float(parsed["calorie_target"][0]) if parsed["calorie_target"][0] is not None else None
                high = float(parsed["calorie_target"][1]) if parsed["calorie_target"][1] is not None else None
                parsed["calorie_target"] = (low, high)
            else:
                parsed.setdefault("calorie_target", (None, None))

            # 1rm: ensure dict of floats
            if "1rm" in parsed and isinstance(parsed["1rm"], dict):
                parsed["1rm"] = {k.strip(): float(v) for k, v in parsed["1rm"].items()}
            else:
                parsed["1rm"] = {}

            return parsed
        except Exception:
            # fall through to heuristic fallback
            llm_response = llm_response  # for debugging if needed
    

In [22]:
### main now accepts a user prompt; LLM decides which tools to call
def main():
    orchestrator = Orchestrator()

    # NOTE: if you previously used CSV loaders or interactive user_profile loader,
    # keep using those. For demo we prompt for user profile here too:
    print("Ready. Enter your user profile now (or press Enter to use defaults).")
    # If you have load_user_profile_from_prompt implemented, you can call it; else use defaults for demo:
    use_defaults = input("Provide us the user profile details like the goal, the calorie target or peak weight you can lift for one complete repetition.").strip().lower()
    try:
        user_profile = load_user_profile_from_prompt(use_defaults)
    except Exception:
        print("Falling back to defaults due to input error.")
        user_profile = {
            "goal": "hypertrophy",
            "calorie_target": (2500, 3000),
            "1rm": {"Barbell Bench Press": 100.0, "Back Squat": 140.0},
        }
    print("User Profile", user_profile)
    # load meals (use CSV loader if present)
    try:
        meals = load_meals_from_csv()
    except Exception:
        # fallback to sample_meals() if CSV not available
        meals = sample_meals()

    # get a user natural-language prompt (what they want the assistant to do)
    user_prompt = input("What would you like me to do? (e.g., 'Make me a workout plan and remind me to drink water'): ").strip()
    user_profile = load_user_profile_from_prompt(user_prompt)
    if not user_prompt:
        print("No prompt provided — running sequential demo.")
        tools = ["sequential"]
    else:
        tools = decide_tools_from_prompt(user_prompt)

    # Run the selected tools via orchestrator dispatcher
    results = orchestrator.run_tools(tools, meals, user_profile)

    # Print results (simple pretty print)
    import json
    print("=== Results ===")
    print(json.dumps(results, indent=2, default=str))

    # metrics
    print_metrics()

if __name__ == "__main__":
    main()

Ready. Enter your user profile now (or press Enter to use defaults).


Provide us the user profile details like the goal, the calorie target or peak weight you can lift for one complete repetition. I want normal workout routine


2025-11-30 15:59:02,435 INFO LLM call 4f424c01 started (len(prompt)=590)


Gemini Call from load_user_profile_from_prompt


2025-11-30 15:59:02,896 INFO AFC is enabled with max remote calls: 10.
2025-11-30 15:59:05,125 INFO HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent "HTTP/1.1 200 OK"
2025-11-30 15:59:05,130 INFO LLM call 4f424c01 finished in 2.695s


...

Gemini Call
Gemini Call


2025-11-30 16:47:42,024 INFO HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent "HTTP/1.1 200 OK"
2025-11-30 16:47:42,028 INFO LLM call 2de409b5 finished in 3.889s
2025-11-30 16:47:42,029 INFO Context compacted from 6897 chars to 572 chars
2025-11-30 16:47:42,030 INFO Attempting Gemini call for WorkoutAgent summary
2025-11-30 16:47:42,210 INFO AFC is enabled with max remote calls: 10.


Gemini Call
Gemini Call


2025-11-30 16:47:46,595 INFO HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent "HTTP/1.1 200 OK"
2025-11-30 16:47:46,599 INFO Gemini summary received (len=471)
2025-11-30 16:47:46,600 INFO METRICS: {'llm_calls': 4, 'errors': 0, 'total_llm_time_s': 14.90760064125061, 'sequential_flows': 0, 'parallel_flows': 0, 'loop_iterations': 0}


Gemini Call
=== Results ===
{
  "workout": {
    "plan": {
      "Day 1 - push": [
        {
          "name": "Push-up",
          "primary": "chest",
          "equipment": "bodyweight",
          "sets": 3,
          "reps": 9,
          "recommended_load_kg": null,
          "notes": [
            "Target intensity: ~62% effort; choose a weight that allows you to hit 9 reps with good form.",
            "Progression: add 2.5-5% load when you hit the top of the rep range for 2 consecutive sessions."
          ]
        },
        {
          "name": "Plank",
          "primary": "core",
          "equipment": "bodyweight",
          "sets": 4,
          "reps": 11,
          "recommended_load_kg": null,
          "notes": [
            "Target intensity: ~74% effort; choose a weight that allows you to hit 11 reps with good form.",
            "Progression: add 2.5-5% load when you hit the top of the rep range for 2 consecutive sessions."
          ]
        },
        {
          "n